# Object Detection on Video Files

- This python notebook loads a video file, gets object assignments as rectangles and tracks.
- This code does not operate on live video files. It is intended to do object detection on video files.
- By default 'trackerCSRT' is selected as CV tracker. Other trackers from  OPENCV_OBJECT_TRACKERS can be selected.



Procedure:
-  1) Select a different trackerName if you like
-  2) Run the code
-  3) Select your video file
-  4) Hit 's' to pause video
-  5) Select an object to be tracked and encapsulate it in a rectangle with mouse
-  6) Hit Space Bar and loop over other objects to be tracked
-  7) Hit 'Esc' to resume video to see live tracking of object

In [ ]:
import cv2
import os
import sys
import imutils
import numpy as np
import tkinter as tk
from tkinter import filedialog
from random import randint

pwd_path = os.getcwd()


# ASK FOR A FILE BY A DIALOG BOX
root = tk.Tk()
root.withdraw()
videoPath = filedialog.askopenfilename()
    #videoPath = pwd_path + '/matchoftheday.mov'
    #videoPath = pwd_path + '/drone.mov'
    #videoPath = pwd_path + '/birds.mov'


# OPENCV_OBJECT_TRACKERS DICTIONARY
OPENCV_OBJECT_TRACKERS = {
    "csrt": cv2.TrackerCSRT_create,
    "kcf": cv2.TrackerKCF_create,
    "boosting": cv2.TrackerBoosting_create,
    "mil": cv2.TrackerMIL_create,
    "tld": cv2.TrackerTLD_create,
    "medianflow": cv2.TrackerMedianFlow_create,
    "mosse": cv2.TrackerMOSSE_create
}

# INITIALISE OpenCV SPECIAL MULTI OBJECT TRACKER
trackers = cv2.MultiTracker_create()
cap = cv2.VideoCapture(videoPath)
trackerName = 'csrt'

while cap.isOpened():

    ret, frame = cap.read()
    
    # Loop out if the video is null
    if frame is None:
        break

    frame = imutils.resize(frame, width=600)
    (success, boxes) = trackers.update(frame)

    # Select box(es) to be tracked on the video
    for box in boxes:
        (x, y, w, h) = [int(v) for v in box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF

    # if the 's' key is selected, we are going to "select" a bounding box to track
    if key == ord("s"):
        colors = []
        # select the enclosing box of the object to be tracked
        # Hit ENTER or SPACE after selecting each target
        box = cv2.selectROIs("Frame", frame, fromCenter=False, showCrosshair=True)
        box = tuple(map(tuple, box)) 
        for bb in box:
            tracker = OPENCV_OBJECT_TRACKERS[trackerName]()
            trackers.add(tracker, frame, bb)

    # Select the 'r' key to reset enclosing box(es)
    elif key == ord("r"):
        trackers.clear()
        trackers = cv2.MultiTracker_create()

        box = cv2.selectROIs("Frame", frame, fromCenter=False,
                            showCrosshair=True)
        box = tuple(map(tuple, box))
        for bb in box:
            tracker = OPENCV_OBJECT_TRACKERS[trackerName]()
            trackers.add(tracker, frame, bb)

    elif key == ord("q"):
        break
        
cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(1)